<a href="https://colab.research.google.com/github/zamirsiddiqui/summarization/blob/master/summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf 
import numpy as np
import nltk
from nltk.corpus import stopwords #provides list of english stopwords
from bs4 import BeautifulSoup
import re
import httpimport

In [70]:
from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import warnings
pd.set_option("display.max_colwidth", 200)
warnings.filterwarnings("ignore")

Using TensorFlow backend.


In [0]:
nltk.download('stopwords')
stop = stopwords.words('english')

In [0]:
# Read files from github
url = 'https://media.githubusercontent.com/media/zamirsiddiqui/summarization/master/train.article.txt'
train_article = pd.read_table(url, error_bad_lines=False, header = None)

In [8]:
pd.set_option('display.max_colwidth', -1)
train_article.head()

,0
0,"australia 's current account deficit shrunk by a record #.## billion dollars -lrb- #.## billion us -rrb- in the june quarter due to soaring commodity prices , figures released monday showed ."
1,"at least two people were killed in a suspected bomb attack on a passenger bus in the strife-torn southern philippines on monday , the military said ."
2,"australian shares closed down #.# percent monday following a weak lead from the united states and lower commodity prices , dealers said ."
3,south korea 's nuclear envoy kim sook urged north korea monday to restart work to disable its nuclear plants and stop its `` typical '' brinkmanship in negotiations .
4,"south korea on monday announced sweeping tax reforms , including income and corporate tax cuts to boost growth by stimulating sluggish private consumption and business investment ."


In [24]:
url = 'https://media.githubusercontent.com/media/zamirsiddiqui/summarization/master/train.title.txt'
train_title = pd.read_table(url, error_bad_lines=False, header = None)
#pd.set_option('display.max_colwidth', -1)
train_title.head()

,0
0,australian current account deficit narrows sharply
1,at least two dead in southern philippines blast
2,australian stocks close down #.# percent
3,envoy urges north korea to restart nuclear disablement
4,skorea announces tax cuts to stimulate economy


In [9]:
train_article.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 1 columns):
0    10000 non-null object
dtypes: object(1)
memory usage: 78.2+ KB


In [0]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",
                           "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",
                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",
                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",
                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",
                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",
                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",
                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",
                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",
                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",
                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",
                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",
                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",
                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",
                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",
                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",
                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",
                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",
                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",
                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",
                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",
                           "you're": "you are", "you've": "you have"}

In [0]:
def text_cleaner(text,num):
    newString = text.lower()
    newString = BeautifulSoup(newString, "lxml").text
    newString = re.sub(r'\([^)]*\)', '', newString)
    newString = re.sub('"','', newString)
    newString = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in newString.split(" ")])    
    newString = re.sub(r"'s\b","",newString)
    newString = re.sub("[^a-zA-Z]", " ", newString) 
    newString = re.sub('[m]{2,}', 'mm', newString)
    if(num==0):
        tokens = [w for w in newString.split() if not w in stop]
    else:
        tokens=newString.split()
    long_words=[]
    for i in tokens:
        if len(i)>1:                                                 #removing short word
            long_words.append(i)   
    return (" ".join(long_words)).strip()

In [0]:
#call the function
cleaned_text = []
for t in train_article[0]:
    cleaned_text.append(text_cleaner(t,0))

In [21]:
cleaned_text[:5]

['australia current account deficit shrunk record billion dollars lrb billion us rrb june quarter due soaring commodity prices figures released monday showed',
 'least two people killed suspected bomb attack passenger bus strife torn southern philippines monday military said',
 'australian shares closed percent monday following weak lead united states lower commodity prices dealers said',
 'south korea nuclear envoy kim sook urged north korea monday restart work disable nuclear plants stop typical brinkmanship negotiations',
 'south korea monday announced sweeping tax reforms including income corporate tax cuts boost growth stimulating sluggish private consumption business investment']

In [25]:
#call the function
cleaned_summary = []
for t in train_title[0]:
    cleaned_summary.append(text_cleaner(t,1))
cleaned_summary[:5]

['australian current account deficit narrows sharply',
 'at least two dead in southern philippines blast',
 'australian stocks close down percent',
 'envoy urges north korea to restart nuclear disablement',
 'skorea announces tax cuts to stimulate economy']

In [0]:
data = pd.DataFrame()
data['cleaned_text']=cleaned_text
data['cleaned_summary']=cleaned_summary

In [63]:
data.head()

,cleaned_text,cleaned_summary
0,australia current account deficit shrunk record billion dollars lrb billion us rrb june quarter due soaring commodity prices figures released monday showed,australian current account deficit narrows sharply
1,least two people killed suspected bomb attack passenger bus strife torn southern philippines monday military said,at least two dead in southern philippines blast
2,australian shares closed percent monday following weak lead united states lower commodity prices dealers said,australian stocks close down percent
3,south korea nuclear envoy kim sook urged north korea monday restart work disable nuclear plants stop typical brinkmanship negotiations,envoy urges north korea to restart nuclear disablement
4,south korea monday announced sweeping tax reforms including income corporate tax cuts boost growth stimulating sluggish private consumption business investment,skorea announces tax cuts to stimulate economy


In [0]:
data.replace('', np.nan, inplace=True)
data.dropna(axis=0,inplace=True)

In [0]:
max_text_len=50
max_summary_len=12

In [0]:
cleaned_text =np.array(data['cleaned_text'])
cleaned_summary=np.array(data['cleaned_summary'])

short_text=[]
short_summary=[]

for i in range(len(cleaned_text)):
    if(len(cleaned_summary[i].split())<=max_summary_len and len(cleaned_text[i].split())<=max_text_len):
        short_text.append(cleaned_text[i])
        short_summary.append(cleaned_summary[i])
        
df=pd.DataFrame({'text':short_text,'summary':short_summary})

In [0]:
df['summary'] = df['summary'].apply(lambda x : 'sostok '+ x + ' eostok')

In [68]:
df.head()

,text,summary
0,australia current account deficit shrunk record billion dollars lrb billion us rrb june quarter due soaring commodity prices figures released monday showed,sostok australian current account deficit narrows sharply eostok
1,least two people killed suspected bomb attack passenger bus strife torn southern philippines monday military said,sostok at least two dead in southern philippines blast eostok
2,australian shares closed percent monday following weak lead united states lower commodity prices dealers said,sostok australian stocks close down percent eostok
3,south korea nuclear envoy kim sook urged north korea monday restart work disable nuclear plants stop typical brinkmanship negotiations,sostok envoy urges north korea to restart nuclear disablement eostok
4,south korea monday announced sweeping tax reforms including income corporate tax cuts boost growth stimulating sluggish private consumption business investment,sostok skorea announces tax cuts to stimulate economy eostok


In [0]:
from sklearn.model_selection import train_test_split
x_tr,x_val,y_tr,y_val=train_test_split(np.array(df['text']),np.array(df['summary']),test_size=0.1,random_state=0,shuffle=True)

In [0]:
from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences

#prepare a tokenizer for reviews on training data
x_tokenizer = Tokenizer() 
x_tokenizer.fit_on_texts(list(x_tr))

In [74]:
thresh=3

cnt=0
tot_cnt=0
freq=0
tot_freq=0

for key,value in x_tokenizer.word_counts.items():
    tot_cnt=tot_cnt+1
    tot_freq=tot_freq+value
    if(value<thresh):
        cnt=cnt+1
        freq=freq+value
    
print("% of rare words in vocabulary:",(cnt/tot_cnt)*100)
print("Total Coverage of rare words:",(freq/tot_freq)*100)

% of rare words in vocabulary: 61.988511453088435
Total Coverage of rare words: 7.796418005633404


In [0]:
#prepare a tokenizer for reviews on training data
x_tokenizer = Tokenizer(num_words=tot_cnt-cnt) 
x_tokenizer.fit_on_texts(list(x_tr))

#convert text sequences into integer sequences
x_tr_seq    =   x_tokenizer.texts_to_sequences(x_tr) 
x_val_seq   =   x_tokenizer.texts_to_sequences(x_val)

#padding zero upto maximum length
x_tr    =   pad_sequences(x_tr_seq,  maxlen=max_text_len, padding='post')
x_val   =   pad_sequences(x_val_seq, maxlen=max_text_len, padding='post')

#size of vocabulary ( +1 for padding token)
x_voc   =  x_tokenizer.num_words + 1

In [76]:
x_voc

5361

In [0]:
#prepare a tokenizer for reviews on training data
y_tokenizer = Tokenizer()   
y_tokenizer.fit_on_texts(list(y_tr))

In [0]:
thresh=4

cnt=0
tot_cnt=0
freq=0
tot_freq=0

for key,value in y_tokenizer.word_counts.items():
    tot_cnt=tot_cnt+1
    tot_freq=tot_freq+value
    if(value<thresh):
        cnt=cnt+1
        freq=freq+value
    
print("% of rare words in vocabulary:",(cnt/tot_cnt)*100)
print("Total Coverage of rare words:",(freq/tot_freq)*100)